In [1]:
# !pip install tensorflow

In [2]:
# !pip install tensorflow_addons

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, PReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score, 
                             mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error)

from tqdm.auto import tqdm

np.random.seed(12345)
tf.random.set_seed(12345)

/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# ========== Data ==========

In [4]:
data = pd.read_csv('dataset.csv')

In [5]:
data

,x1,x2,y
0,2,1,9
1,2,-1,1
2,-2,2,6
3,1,2,11
4,-2,3,9
5,2,0,5
6,-1,-1,-2
7,-2,1,3
8,0,0,2
9,1,-1,0


In [6]:
X = data[['x1', 'x2']]

In [7]:
y = data['y']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

# ========== Model ==========

In [9]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [10]:
devices_type = [device.device_type for device in tf.config.list_physical_devices()]

devices_type

['CPU']

In [11]:
if 'GPU' in devices_type:
    device_name = '/device:GPU:0'
else:
    device_name = '/device:CPU:0'

In [12]:
best_mse = np.inf

l1_ratios = [0.01, 0.5, 0.75, 1]
alphas = np.logspace(-3, 2, 100)

for l1_ratio in tqdm(l1_ratios):
    for alpha in tqdm(alphas):
        l1 = alpha*l1_ratio
        l2 = alpha*(1-l1_ratio)
        model = Sequential()

        model.add(Dense(5, input_dim=X.shape[1], activation='relu', 
                        kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), 
                        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        model.add(Dense(4, activation='relu', 
                        kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), 
                        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        model.add(Dense(1, activation=None, 
                        kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), 
                        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))

        model.compile(loss=MeanSquaredError(), optimizer=SGD(learning_rate=0.01))

        tqdm_callback = tfa.callbacks.TQDMProgressBar(show_epoch_progress=False)

        with tf.device(device_name):
            history = model.fit(X_train, y_train, epochs=500, callbacks=[tqdm_callback], verbose=0)  

        y_pred_val = model.predict(X_val)
        if np.isnan(np.min(y_pred_val)):
            continue
        mse = mean_squared_error(y_val, y_pred_val)

        if mse < best_mse:
            best_mse = mse
            best_model = model
            best_l1_ratio = l1_ratio
            best_alpha = alpha

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

...

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

In [13]:
best_l1_ratio

0.75

In [14]:
best_alpha

0.0015922827933410922

In [15]:
best_model.get_weights()

[array([[ 0.88009214, -0.5337207 , -0.0291096 ,  0.21009769,  0.5901478 ],
        [-0.20323794, -0.1451744 , -0.54650724,  0.07612803,  0.45981175]],
       dtype=float32),
 array([ 0.02903866,  0.54758114,  1.7600471 , -0.16764958, -0.4784884 ],
       dtype=float32),
 array([[-0.62448514,  0.39843094,  0.46970168,  0.12378173],
        [ 0.2709263 ,  0.4487759 ,  0.64062965,  0.1375103 ],
        [ 1.5599197 ,  0.9066388 ,  0.41072306,  0.77791345],
        [-0.16666141, -0.6741056 ,  0.31271037,  0.07918659],
        [-0.18729976, -0.69564223, -0.75002897, -0.46852875]],
       dtype=float32),
 array([0.3578252 , 0.6505335 , 0.37576336, 0.29300556], dtype=float32),
 array([[-1.6199988],
        [-1.3531165],
        [-0.8759001],
        [-1.0278802]], dtype=float32),
 array([13.670932], dtype=float32)]

# ========== Prediction ==========

In [16]:
X = pd.read_csv('incoming_data.csv')

In [17]:
X

,x1,x2
0,0,3
1,-2,-1
2,0,1
3,0,-1


In [18]:
best_model.predict(X)

array([[12.8345585],
       [-3.11584  ],
       [ 4.91467  ],
       [-0.9786024]], dtype=float32)